In [ ]:
import gc
import datetime
from fastai.tabular import *
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_log_error as msle, mean_squared_error as mse
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Fill missing values with median 
def FillMissing(df):
  for col in df.columns:
    if pd.isnull(df[col]).sum():
      filler = df[col].median()
      df[col] = df[col].fillna(filler)

In [ ]:
# Predictions lower than zero are turned zero
def fix_predictions(y):
    """
    :param y: Column with predictions
    """
    y[y < 0] = 0

In [ ]:
# Predictibility in our predictions
def seed_everything(seed):
    """
    :param seed: Value for seeding random functions
    """
    random.seed(seed)
    np.random.seed(seed)

In [ ]:
seed = 5
seed_everything(seed)

In [ ]:
# Reducing memory usage, based on: https://www.kaggle.com/bwilsonkg/column-statistics
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

""" 
Evaluation metric used in this competition: 
https://www.kaggle.com/c/ashrae-energy-prediction/overview/evaluation
""""
def rmsle(y_true: pd.Series, y_predict: pd.Series) -> float:
    """
    Evaluate root mean squared log error
    :param y_true:
    :param y_predict:
    :return:
    """
    return np.sqrt(msle(y_true, y_predict))

In [ ]:
# We need this to download datasets from kaggle 
! {sys.executable} -m pip install kaggle --upgrade

In [ ]:
# Upload your credentials. You can donwload the API token from your kaggle account
! mkdir -p ~/.kaggle/
! mv kaggle.json ~/.kaggle/

In [ ]:
path = Config.data_path()/'ASHRAE'
path.mkdir(parents=True, exist_ok=True)
path

In [ ]:
# Download data competition 
! kaggle competitions download -c ashrae-energy-prediction -p {path}

In [ ]:
# Unzip files
! unzip -q -n {path}/'ashrae-energy-prediction.zip' -d {path}

In [ ]:
"""" 
Read data to df. Note that we are using only 
training data in this notebook. 
We will use the test data for our predicitons 
and submission later in a separated notebook.
""""
build = reduce_mem_usage(pd.read_csv(path/'building_metadata.csv'))
train = reduce_mem_usage(pd.read_csv(path/'train.csv'))
weather_train = reduce_mem_usage(pd.read_csv(path/'weather_train.csv'))

In [ ]:
FillMissing(train)
FillMissing(build)
FillMissing(weather_train)

In [ ]:
""""
We perform a lag feature transformation.
Lag features are the classical way that 
time series forecasting problems are 
transformed into supervised learning problems.
Further reading:
https://machinelearningmastery.com/basic-feature-engineering-time-series-data-python/
""""
def add_lag_feature(weather_df, window=3):
    group_df = weather_df.groupby('site_id')
    cols = ['air_temperature'] 
    rolled = group_df[cols].rolling(window=window, min_periods=0)
    lag_mean = rolled.mean().reset_index().astype(np.float16)
    lag_max = rolled.max().reset_index().astype(np.float16)
    lag_min = rolled.min().reset_index().astype(np.float16)
    lag_std = rolled.std().reset_index().astype(np.float16)
    for col in cols:
        weather_df[f'{col}_mean_lag{window}'] = lag_mean[col]
        weather_df[f'{col}_max_lag{window}'] = lag_max[col]
        weather_df[f'{col}_min_lag{window}'] = lag_min[col]
        weather_df[f'{col}_std_lag{window}'] = lag_std[col]


In [ ]:
""""
Transform some data into a more meaningful feature.
For example we transform the wind speed into Beaufort
scale: https://es.wikipedia.org/wiki/Escala_de_Beaufort,
or wind direction into compass direction:
https://en.wikipedia.org/wiki/Points_of_the_compass
""""
def prepare_data(train,build,weather):
   
    add_lag_feature(weather, window=72)
    train = train.merge(build, on='building_id',how='left')  
    train = train.merge(weather, on=['site_id', 'timestamp'], how='left')
    train['timestamp'] = pd.to_datetime(arg=train['timestamp'])
        
    train['year'] = train['timestamp'].dt.year
    train['month'] = train['timestamp'].dt.month
    train['day'] = train['timestamp'].dt.day
    train['hour'] = train['timestamp'].dt.hour
    train['weekday'] = train['timestamp'].dt.weekday
    
    train = reduce_mem_usage(train)

    beaufort = [(0, 0, 0.3), (1, 0.3, 1.6), (2, 1.6, 3.4), (3, 3.4, 5.5), (4, 5.5, 8), (5, 8, 10.8), (6, 10.8, 13.9), 
          (7, 13.9, 17.2), (8, 17.2, 20.8), (9, 20.8, 24.5), (10, 24.5, 28.5), (11, 28.5, 33), (12, 33, 200)]
    
    for item in beaufort:
        train.loc[(train['wind_speed']>=item[1]) & (train['wind_speed']<item[2]), 'beaufort_scale'] = item[0]
    
    #oktas = [('SKC', 0.0, 0.0), ('FEW', 1.0, 2.0), ('SCT', 3.0, 4.0), ('BKN', 5.0, 7.0), ('OVC', 8.0, 8.0), ('SOV', 9.0, 9.0)]

    #for item in oktas:
      #train.loc[(train['cloud_coverage']==item[1] and item[2]) | ((train['cloud_coverage']>=item[1]) & (train['cloud_coverage']<=item[2])),'oktas_scale'] = item[0]

    compass = [('N', 360, 0), ('NbE', 0, 16.875), ('NNE', 16.875, 28.125), ('NEbN', 28.125, 39.375), ('NE', 39.375, 50.625), 
              ('NEbE', 50.625, 61.875), ('ENE', 61.875, 73.125), ('EbN', 73.125, 84.375), ('E', 84.375, 95.625), ('EbS', 95.625, 106.875),
              ('ESE', 106.875, 118.125), ('SEbE', 118.125, 129.375), ('SE', 129.375, 140.625), ('SEbS', 140.625, 151.875), ('SSE', 151.875, 163.125),
              ('SbE', 163.125, 174.375), ('S',174.375, 185.625), ('SbW', 185.625, 196.875), ('SSW', 196.875, 208.125), ('SWbS', 208.125, 219.375), 
              ('SW', 219.375, 230.625), ('SWbW', 230.625, 241.875), ('WSW', 241.875, 253.125), ('WbS', 253.125, 264.375), ('W', 264.375, 275.625), 
              ('WbN', 275.625, 286.875), ('WNW', 286.875, 298.125), ('NWbW', 298.125, 309.375), ('NW', 309.375, 320.625), ('NWbN', 320.625, 331.875), 
              ('NNW', 331.875, 343.125), ('NbW', 343.125, 360)]

    for item in compass:  
      train.loc[(train['wind_direction']>=item[1]) & (train['wind_direction']<item[2]), 'compass_direction'] = item[0]

    comfort = [('Pleasant', -35.0, 12.7778), ('Comfy', 12.7778 , 15.5556), ('Sticky', 15.5556, 18.3333), ('Uncomfy', 18.3333, 21.1111), 
              ('Oppressive', 21.1111, 23.8889), ('Miserable', 23.8889, 26.1)] 
    
    for item in comfort:
      train.loc[(train['dew_temperature']>=item[1]) & (train['dew_temperature']<item[2]), 'comfort'] = item[0]
    
    train['age'] = train['year'] - train['year_built']
    train = reduce_mem_usage(train)

    return train
    #if not test:
        #X = X.query('not(site_id==0 & timestamp<"2016-05-21 00:00:00")')
        

In [ ]:
train = prepare_data(train,build,weather_train)

In [ ]:
# Delete outliers as we decide in our previous EDA notebook
train = train[train.building_id != 778] 
train = train[train.building_id != 1088] 

In [ ]:
# Check our data in dataframe
train

In [ ]:
# Defining our variables(features) as continous or categoricals and prediction target.
numericals = ["square_feet", 'precip_depth_1_hr','air_temperature_mean_lag72', 'air_temperature_max_lag72', 'primary_use' 
              'air_temperature_min_lag72', 'air_temperature_std_lag72', 'site_id', 'building_id', 'cloud_coverage', 'age']

categoricals = ["hour", "weekday", "meter",'comfort','floor_count', 'month', 'compass_direction'] #'oktas_scale'
target = 'meter_reading'
features = categoricals + numericals

In [ ]:
# Transform categoricals features into category
def Type(train):
 for col in train[categoricals]: 
  train[col] = train[col].astype('category')

In [ ]:
# Log + 1 transform of target input
train[target] = np.log1p(train[target])

In [ ]:
"""
Further information of Cross-Validation methods:
https://machinelearningmastery.com/k-fold-cross-validation/
"""

In [1]:
"""
XGB Parameters see lightgbm documentation:
https://xgboost.readthedocs.io/en/latest/parameter.html"""
params = {
    'min_child_weight': 10.0,
    'objective': 'reg:squaredlogerror',
    'max_depth': 7,
    'max_delta_step': 1.8,
    'colsample_bytree': 0.4,
    'subsample': 0.8,
    'eta': 0.025,
    'gamma': 0.65,
    'num_boost_round' : 700 
    }

**KStratfold**

In [ ]:
folds = 2
kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

for train_index, val_index in kf.split(train, train['building_id']):
  train_X = train[features].iloc[train_index]
  val_X = train[features].iloc[val_index]
  train_y = train[target].iloc[train_index]
  val_y = train[target].iloc[val_index]
  xgb_train = xgb.DMatrix(train_X, train_y)
  xgb_eval = xgb.DMatrix(val_X, val_y)
  gbm = xgb.train(params,
                 xgb_train,
                 num_boost_round=2500,
                 valid_sets=(xgb_train, xgb_eval),
                 early_stopping_rounds=150,
                 verbose_eval = 150)

In [ ]:
gbm.save_model('lgb_classifier_{}_{}.txt'.format(datetime.datetime.now().strftime("%d-%m-%Y"), rmsle), num_iteration=gbm.best_iteration)